# READ ME

- selenium을 활용하여 일자 별 총 관객수를 구하는 코드입니다.
- 다운 받을 때 파일명이 동일합니다. 클릭을 누른 날을 기준으로 'KOBIS_총_관객수_및_매출액_일별_2023-04-07'와 같이 저장되는데요
- 파일명이 동일하기 때문에 일자별로 구분을 해주어야 하는데, 사본 형식으로 저장이됩니다.
    - KOBIS_총_관객수_및_매출액_일별_2023-04-07(1)
    - KOBIS_총_관객수_및_매출액_일별_2023-04-07(2) 
- 따라서, 이를 구분해줄 필요가 있어, 저장 후 이름을 변경하여 새 경로로 이동하는 코드를 짜주었습니다.
- 또한, xls파일로 저장이되어서, 이를 xlsx로 변환시켜주는 과정을 코드에 넣었습니다.
----

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager    # 매 번 크롬 드라이버를 설치할 필요없이 자동으로
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from tqdm import tqdm_notebook
from urllib.request import urlopen
import time
import requests

driver = webdriver.Chrome("chromedriver")
driver.get("https://www.kobis.or.kr/kobis/business/stat/them/findDailyTotalList.do")

year_element = driver.find_element_by_id("selectYear")
month_element = driver.find_element_by_id("selectMonth")

# 2023년~2004 년은 option 태그로 묶여있기 때문에, Xpath로 접근 -> 2023년부터 2004년까지 총 option이 몇개 있는지 길이를 알기 위해 형성.
year_range = year_element.find_elements_by_tag_name("option")    
month_range = month_element.find_elements_by_tag_name("option")


options = webdriver.ChromeOptions()


try:
    for i in range(1,len(year_range)+1):
        search_xpath = '//*[@id="content"]/div[3]/div/a'                                            # 엑셀 다운 받는 xpath
        driver.find_element(By.XPATH, '//*[@id="selectYear"]/option[{}]'.format(i)).click()         # 년 선택
        for j in range(1,len(month_range)+1):
            driver.find_element(By.XPATH, '//*[@id="selectMonth"]/option[{}]'.format(j)).click()    # 월 선택
            driver.find_element(By.CLASS_NAME, "btn_blue").click()                                  # 조회버튼 클릭
            
            driver.find_element(By.CLASS_NAME, "btn_small").click()                                 # 엑셀다운버튼 클릭.
        
            
            file_name = "{}년 {}월총 관객 수 .xls".format(i,j)

            search_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, search_xpath)))                               # 엑셀 다운 받고 대기
            
            
            time.sleep(2)
            
            # 기존에 다운받아지는 폴더의 경로
            file_path = 'C:\\Users\\user\\Downloads\\KOBIS_총_관객수_및_매출액_일별_2023-04-07.xls' 
            
            # 파일의 이름 설정을 위한 코드
            new_file_name = 'KOBIS_일자별_통계정보 {}년 {}월.xls'.format(2024-i,j)              
            
            # 다운 받은 파일들을 저장할 폴더
            new_file_path = 'C:\\js_acorn\\Database\\pythonwork\\Analysis\\Movie_data_raw\\' + new_file_name 

            shutil.move(file_path, new_file_path)        # 다운받아진 폴더의 경로에서 내가 원하는 경로로 파일을 옮겨줌.
        
except Exception as e:
    print(e)
    

import os
import win32com.client as win32


# xls 파일을 깨지지 않고 xlsx로 변환시키는 코드
def convert_xls_to_xlsx(xls_file_path, xlsx_file_path):
    excel = win32.Dispatch("Excel.Application")
    excel.DisplayAlerts = False
    wb = excel.Workbooks.Open(xls_file_path)
    wb.SaveAs(xlsx_file_path, FileFormat=51)  # FileFormat=51 corresponds to .xlsx
    wb.Close()
    excel.Quit()

input_folder = 'C:\\js_acorn\\Database\\pythonwork\\Analysis\\Movie_data_raw'    # xls파일들이 모여있는 폴더의 경로를 지정해주세요.
output_folder = 'C:\\js_acorn\\Database\\pythonwork\\Analysis\\Movie_data'       # xlsx로 변환된 파일이 저장될 경로를 지정해주세요.

for file_name in os.listdir(input_folder):
    if file_name.endswith('.xls'):
        input_file_path = os.path.join(input_folder, file_name)
        output_file_name = file_name.replace('.xls', '.xlsx')
        output_file_path = os.path.join(output_folder, output_file_name)

        print(f"Converting {input_file_path} to {output_file_path}")
        convert_xls_to_xlsx(input_file_path, output_file_path)
        print(f"Conversion complete for {output_file_path}")

name 'shutil' is not defined
Converting C:\js_acorn\Database\pythonwork\Analysis\Movie_data_raw\KOBIS_일자별_통계정보 2004년 10월.xls to C:\js_acorn\Database\pythonwork\Analysis\Movie_data\KOBIS_일자별_통계정보 2004년 10월.xlsx
Conversion complete for C:\js_acorn\Database\pythonwork\Analysis\Movie_data\KOBIS_일자별_통계정보 2004년 10월.xlsx
Converting C:\js_acorn\Database\pythonwork\Analysis\Movie_data_raw\KOBIS_일자별_통계정보 2004년 11월.xls to C:\js_acorn\Database\pythonwork\Analysis\Movie_data\KOBIS_일자별_통계정보 2004년 11월.xlsx
Conversion complete for C:\js_acorn\Database\pythonwork\Analysis\Movie_data\KOBIS_일자별_통계정보 2004년 11월.xlsx
Converting C:\js_acorn\Database\pythonwork\Analysis\Movie_data_raw\KOBIS_일자별_통계정보 2004년 12월.xls to C:\js_acorn\Database\pythonwork\Analysis\Movie_data\KOBIS_일자별_통계정보 2004년 12월.xlsx


KeyboardInterrupt: 

# 저장된 파일들을 데이터 프레임으로 읽어줍니다.

In [6]:
#### 저장된 파일들을 df로 읽어오기. 

dfs = []      # dfs라는 빈 리스트를 만들어주고, 파일들을 불러와서 리스트에 추가
for i in tqdm(range(2004,2024)): 
    for j in range(1,13):
        movie_df = pd.read_excel('C:\\js_acorn\\Database\\pythonwork\\Analysis\\Movie_data\\KOBIS_일자별_통계정보 {}년 {}월.xlsx'.format(i,j),
                                        sheet_name = 'KOBIS_일자별_통계정보 {}년 {}월'.format(i,j), 
                                        header = [4,5])
        dfs.append(movie_df)

#### 데이터 프레임의 컬럼들이 모두 같기 때문에, 행 방향으로 concat 해주기
total_audience = pd.DataFrame()      

for df in tqdm(dfs):
    total_audience = pd.concat([total_audience, df],ignore_index=True)  # 인덱스 무시하고 합치기.
    
    
total_audience.to_csv("./data/Total_audience.csv", encoding="utf-8")   # 완성된 데이터프레임을 csv파일로 저장. 경로 설정해주기.
    


C:\Users\user\AppData\Local\Temp\ipykernel_15176\2408246183.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(2004,2024)):


  0%|          | 0/20 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_15176\2408246183.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for df in tqdm_notebook(dfs):


  0%|          | 0/240 [00:00<?, ?it/s]